In [74]:
import numpy as np
from susy_qm import calculate_Hamiltonian, calculate_wz_hamiltonian
import pennylane as qml
from pennylane import numpy as pnp
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize

In [14]:
cutoff = 4
potential = 'QHO'

In [15]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)
eigenvalues, eigenvectors = np.linalg.eig(H)

min_index = np.argmin(eigenvalues)
min_eigenvalue = eigenvalues[min_index]
min_eigenvector = eigenvectors[:, min_index]

In [16]:
#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [36]:
params = pnp.array([[
            [2.195201075128017,
            3.106916574276913,
            4.187261286195278],
            [3.046766119144531,
            0.025863111078491485,
            3.146307109203438],
            [0.5004353269790704,
            6.258003111574653,
            2.376347569714709]
        ]],  requires_grad=True)

In [73]:
# Define the PennyLane device
dev = qml.device('default.qubit', wires=num_qubits, shots=None)

# Define the parameterized ansatz
@qml.qnode(dev)
def ansatz(params):
    qml.StronglyEntanglingLayers(params, wires=range(num_qubits), imprimitive=qml.CZ)
    return qml.state()  # Return the statevector

# Initialize random parameters for the ansatz
num_layers = 1
params_shape = qml.StronglyEntanglingLayers.shape(n_layers=num_layers, n_wires=num_qubits)
#params = np.random.random(params_shape)

# Get the statevector from the ansatz
ansatz_state = ansatz(params)

# Compute the overlap
overlap = np.vdot(min_eigenvector, ansatz_state)  # Inner product
overlap_squared = np.abs(overlap)**2  # Overlap squared

print("Overlap:", overlap)
print("Overlap squared:", overlap_squared)


Overlap: (0.3193184491593054-0.947647469992294j)
Overlap squared: 0.9999999993562998


In [67]:
def cost_function(params):

    params = pnp.tensor(params.reshape(params_shape), requires_grad=True)
    ansatz_state = ansatz(params)

    overlap = np.vdot(min_eigenvector, ansatz_state)
    overlap_squared = np.abs(overlap)**2  

    return (1 - overlap_squared)

In [68]:
x0 = np.random.random(params_shape).flatten()

res = minimize(
    cost_function,
    x0,
    method= "COBYLA",
    options= {'maxiter':10000}
)


In [69]:
res.fun

np.float64(6.437002042503082e-10)